In [2]:
import sys
sys.path.insert(0,"/Users/chenry/code/ModelSEEDpy")
sys.path.append("/Users/chenry/code/cobrakbase")
#sys.path.append("/Users/piehl/Desktop/S2022/ModelSEEDpy-main/modelseedpy")
#sys.path.insert(0,"/Users/piehl/Desktop/S2022/ModelSEEDpy")
#sys.path.append("/Users/piehl/Desktop/S2022/cobrakbase-cobra-model")
import os
#os.environ["HOME"] = "/Users/piehl/Desktop/S2022"
import json
import cobra
import cobrakbase
from modelseedpy import MSPackageManager, MSGrowthPhenotypes
from cobrakbase.core.kbasefba.fbamodel_from_cobra import CobraModelConverter
import pandas as pd
from IPython.display import HTML

cobrakbase 0.3.1


In [2]:
#Read in biolog data
zpheno_list = []
bpheno_list = []
data = pd.read_csv("BiologMacTestOutput_On-Off.csv")
bdata = {}
for index, row in data.iterrows():
    media = row["Media"]
    if media[0:7] == "Carbon-":#Comment this out if you want to use all media
        bpheno_list.append(media)
        for (columnName, columnData) in data.iteritems():
            if columnName != "Media":
                genome_id = columnName[0:-9]
                if genome_id not in bdata:
                    bdata[genome_id] = {}    
                bdata[genome_id][media] = row[columnName]
#Read in metadata sheet and use to translate genome IDs
#Deal with trailing zeros and load and save all metadata
metadata = {}
translation = {}
data = pd.read_csv("Watershed Metadata 08_23_16-present - PatricMasterSheet 1.csv")
for index, row in data.iterrows():
    genome_id = str(row["Patric ID"])
    for i in range(3):
        if genome_id in bdata:
            break
        genome_id += "0"
    if genome_id not in bdata:
        print(str(row["Patric ID"])+";not found")
        next
    local_genome = row["Local Genome ID from Patric"]
    local_genome = local_genome[17:]
    translation[local_genome] = genome_id
    metadata[genome_id] = row
#Now making sure all genomes from clayton are in metadata
for genome_id in bdata:
    if genome_id not in metadata:
        print(genome_id+";no metadata")
#Read in biolog predictions from classifiers
data = pd.read_csv("Aaron_predict_178.csv")
zdata = {}
for index, row in data.iterrows():
    genome = row["genome"]
    if genome[-4:] == ".txt":
        genome = genome[:-4]
    if genome not in translation:
        print(genome+";no translation")
    else:
        genome = translation[genome]
        if genome not in zdata:
            zdata[genome] = {}
        for (columnName, columnData) in data.iteritems():
            if columnName != "genome":
                if columnName not in zpheno_list:
                    zpheno_list.append(columnName)
                zdata[genome][columnName] = row[columnName]  

562.8594;not found
562.85941;not found
562.81369;not found
562.81371;not found
562.81372;not found
562.81363;not found
562.85952;not found
562.85953;not found
562.85942;not found
562.85943;not found
562.85944;not found
562.85945;not found
562.85946;not found
562.85947;not found
562.85948;not found
562.85949;not found
562.8595;not found
562.85951;not found
562.55367;no metadata
562.55368;no metadata
562.55380;no metadata
562.55381;no metadata
562.55382;no metadata
562.55383;no metadata
562.55384;no metadata
562.55385;no metadata
562.55386;no metadata
562.55390;no metadata
562.55391;no metadata
562.55413;no metadata
562.55415;no metadata
562.55417;no metadata
562.55418;no metadata
562.55483;no metadata
562.55540;no metadata
562.55545;no metadata
562.55546;no metadata
562.55547;no metadata
562.55548;no metadata
562.55550;no metadata
562.55551;no metadata
562.55552;no metadata
562.55554;no metadata
562.55556;no metadata
562.55557;no metadata
562.55559;no metadata
562.55561;no metadata
562.

In [3]:
mdlws = 119455
#kbase_api = cobrakbase.KBaseAPI()
kbase_api = cobrakbase.KBaseCache(token=token,dev=True)
models = kbase_api.list_objects(mdlws, object_type="KBaseFBA.FBAModel",include_metadata=True)
model_indecies = {}
model_names = []
modelrxnlists = {}
reaction_hash = {}
for model in models:
    model_indecies[model[1]] = len(model_names)
    model_names.append(model[1])
    modelobj = kbase_api.get_object(model[1],mdlws)
    reactionlist = modelobj['modelreactions']
    modelrxnlists[model[1]] = []
    for reaction in reactionlist:
        if reaction['id'] not in reaction_hash:
            reaction_hash[reaction['id']] = reaction
        if 'modelReactionProteins' in reaction.keys() and len(reaction['modelReactionProteins']):
            modelrxnlists[model[1]].append(reaction['id'])
reaction_distances = [[0 for i in range(len(models))] for j in range(len(models))]
biolog_distances = [[0 for i in range(len(models))] for j in range(len(models))]
classifier_distances = [[0 for i in range(len(models))] for j in range(len(models))]
consolidated_distances = [[0 for i in range(len(models))] for j in range(len(models))]
count = 0
conflicts = {}
for modelone in models:
    conflicts[modelone[1][0:-9]] = 0
    counttwo = 0
    for zpheno in zpheno_list:
        if modelone[1][0:-9] in zdata and zdata[modelone[1][0:-9]][zpheno] > 0.001:
            if modelone[1][0:-9] in bdata and bdata[modelone[1][0:-9]][zpheno] < 0.001:
                conflicts[modelone[1][0:-9]] += 1
        elif modelone[1][0:-9] in bdata and bdata[modelone[1][0:-9]][zpheno] > 0.001:
            conflicts[modelone[1][0:-9]] += 1
    for modeltwo in models:
        for reaction in reaction_hash.keys():
            if reaction in modelrxnlists[modelone[1]]:
                if reaction not in modelrxnlists[modeltwo[1]]:
                    reaction_distances[count][counttwo] += 1
                    reaction_distances[counttwo][count] += 1
            if reaction in modelrxnlists[modeltwo[1]]:
                if reaction not in modelrxnlists[modelone[1]]:
                    reaction_distances[count][counttwo] += 1
                    reaction_distances[counttwo][count] += 1
        for phenotype in bpheno_list:
            if modelone[1][0:-9] in bdata and bdata[modelone[1][0:-9]][phenotype] > 0.001:
                if modeltwo[1][0:-9] in bdata and bdata[modeltwo[1][0:-9]][phenotype] < 0.001:
                    biolog_distances[count][counttwo] += 1
                    biolog_distances[counttwo][count] += 1
            if modeltwo[1][0:-9] in bdata and bdata[modeltwo[1][0:-9]][phenotype] > 0.001:
                if modelone[1][0:-9] in bdata and bdata[modelone[1][0:-9]][phenotype] < 0.001:
                    biolog_distances[count][counttwo] += 1
                    biolog_distances[counttwo][count] += 1
        for zpheno in zpheno_list:    
            if modelone[1][0:-9] in zdata and zdata[modelone[1][0:-9]][zpheno] > 0.001:
                if modeltwo[1][0:-9] in zdata and zdata[modeltwo[1][0:-9]][zpheno] < 0.001:
                    classifier_distances[count][counttwo] += 1
                    classifier_distances[counttwo][count] += 1
            if modeltwo[1][0:-9] in zdata and zdata[modeltwo[1][0:-9]][zpheno] > 0.001:
                if modelone[1][0:-9] in zdata and zdata[modelone[1][0:-9]][zpheno] < 0.001:
                    classifier_distances[count][counttwo] += 1
                    classifier_distances[counttwo][count] += 1
        counttwo += 1
    count += 1
with open('ReactionDist.json', 'w') as outfile:
    json.dump(reaction_distances, outfile)
with open('BiologDist.json', 'w') as outfile:
    json.dump(biolog_distances, outfile)
with open('ClassifierDist.json', 'w') as outfile:
    json.dump(classifier_distances, outfile)   
largest_rxn = None
largest_class = None
largest_biolog = None
count = 0
for modelone in models:
    counttwo = 0
    for modeltwo in models:
        if not largest_rxn or reaction_distances[count][counttwo] > largest_rxn:
            largest_rxn = reaction_distances[count][counttwo]
        if not largest_class or classifier_distances[count][counttwo] > largest_class:
            largest_class = classifier_distances[count][counttwo]
        if not largest_biolog or biolog_distances[count][counttwo] > largest_biolog:
            largest_biolog = biolog_distances[count][counttwo]
        counttwo += 1
    count += 1
count = 0
for modelone in models:
    counttwo = 0
    for modeltwo in models:
        consolidated_distances[count][counttwo] = reaction_distances[count][counttwo]/largest_rxn + classifier_distances[count][counttwo]/largest_class + biolog_distances[count][counttwo]/largest_biolog
#Cluster on consolidated and all individuals
#Print cluster size profile for all
#Map clusters for each method and compute similarity and show average similarity + distribution
#Print cluster representatives for each method
#Print metadata for clusters
#Compute average and range of consistency values for each cluster
#Pick representative genome with the highest inconsistency between classifiers and models

In [4]:
with open('ConsolidatedDist.json', 'w') as outfile:
    json.dump(consolidated_distances, outfile)   

In [4]:
mdlws = 119455
#kbase_api = cobrakbase.KBaseAPI()
kbase_api = cobrakbase.KBaseCache()
models = kbase_api.list_objects(mdlws, object_type="KBaseFBA.FBAModel",include_metadata=True)
model_indecies = {}
model_names = []
modelrxnlists = {}
reaction_hash = {}
for model in models:
    model_names.append(model[1])
with open('ModelIndecies.json', 'w') as outfile:
    json.dump(model_names, outfile)  